<a href="https://colab.research.google.com/github/rociavl/UNI/blob/main/P2_OP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import networkx as nx
import matplotlib.pyplot as plt

# Definir la estructura de precedencias y tiempos
precedencias_proyecto = {
    'A': [], 'B': ['A'], 'C': ['B', 'H'], 'D': [], 'E': ['G'], 'F': ['E'],
    'G': ['D'], 'H': ['G'], 'I': ['D'], 'J': ['I'], 'K': ['D'],
    'L': ['J', 'K'], 'M': ['C', 'L', 'F'], 'O': ['M'], 'P': ['M'],
    'Q': ['T'], 'R': ['P'], 'S': ['O'], 'T': ['R', 'S'], 'U': ['Q']
}

# Tiempos de duración de las actividades
tiempos_proyecto = {
    'A': 4, 'B': 3, 'C': 2, 'D': 7, 'E': 2, 'F': 10, 'G': 5,
    'H': 15, 'I': 5, 'J': 6, 'K': 10, 'L': 2, 'M': 3, 'O': 8,
    'P': 5, 'Q': 2, 'R': 4, 'S': 3, 'T': 6, 'U': 5
}

# Crear el grafo dirigido
G_proyecto = nx.DiGraph()

# Agregar un nodo ficticio inicial
nodo_inicial_ficticio = 'α'
for tarea, predecesores in precedencias_proyecto.items():
    if not predecesores:  # Si la tarea no tiene predecesores
        G_proyecto.add_edge(nodo_inicial_ficticio, tarea, weight=0)  # Conectar al nodo ficticio con peso 0
    for predecesor in predecesores:
        G_proyecto.add_edge(predecesor, tarea, weight=tiempos_proyecto[tarea])

# Añadir nodo ficticio final
nodo_final_ficticio = 'ω'
for tarea in list(G_proyecto.nodes):  # Convertimos los nodos a una lista para evitar cambios durante la iteración
    if G_proyecto.out_degree(tarea) == 0 and tarea != nodo_inicial_ficticio:  # Si no tiene sucesores
        G_proyecto.add_edge(tarea, nodo_final_ficticio, weight=0)  # Conectar al nodo ficticio final con peso 0

# Añadir tiempos de procesamiento como atributos de nodos
nx.set_node_attributes(G_proyecto, tiempos_proyecto, 'duration')

# Utilizar nx.dag_longest_path para calcular el camino crítico
camino_critico = nx.dag_longest_path(G_proyecto, weight='weight')

# Filtrar nodos ficticios
camino_critico = [nodo for nodo in camino_critico if nodo not in [nodo_inicial_ficticio, nodo_final_ficticio]]

# Duración total del proyecto (longitud del camino crítico)
duracion_total_proyecto = sum(tiempos_proyecto[nodo] for nodo in camino_critico)
duracion_total_2 = nx.dag_longest_path_length(G_proyecto, weight='weight')
# Imprimir resultados
print(f"1. Duración mínima del proyecto: {duracion_total_proyecto} días")
print(f"2. Camino Crítico: {camino_critico}")

# Calcular Early Start (ES) y Early Finish (EF)
early_start = {nodo: 0 for nodo in G_proyecto.nodes}
for nodo in nx.topological_sort(G_proyecto):  # Orden topológico
    for predecesor in G_proyecto.predecessors(nodo):
        early_start[nodo] = max(early_start[nodo], early_start[predecesor] + tiempos_proyecto.get(predecesor, 0))

early_finish = {nodo: early_start[nodo] + tiempos_proyecto.get(nodo, 0) for nodo in G_proyecto.nodes}

# Duración total del proyecto es el máximo tiempo de finalización
duracion_total_proyecto = max(early_finish.values())

# Calcular Late Finish (LF) y Late Start (LS) hacia atrás
late_finish = {nodo: duracion_total_proyecto for nodo in G_proyecto.nodes}
for nodo in reversed(list(nx.topological_sort(G_proyecto))):  # Invertimos el orden topológico
    for sucesor in G_proyecto.successors(nodo):
        late_finish[nodo] = min(late_finish[nodo], late_finish[sucesor] - tiempos_proyecto.get(sucesor, 0))

late_start = {nodo: late_finish[nodo] - tiempos_proyecto.get(nodo, 0) for nodo in G_proyecto.nodes}

# Calcular holguras (slack)
margenes = {nodo: late_start[nodo] - early_start[nodo] for nodo in G_proyecto.nodes}

# Sumar los márgenes de las actividades C, G, L y S
margen_total = sum(margenes[nodo] for nodo in ['C', 'G', 'L', 'S'])
print(f"3. Suma de los márgenes de las actividades C, G, L y S: {margen_total}, Margenes:{margenes} ")

# Cambio G->I o  G <-I
def camino_nuevo(node1, node2):
    # Cálculo de los tiempos máximos al cambiar la relación entre node1 y node2
    D_max = {
        'Right': max(duracion_total_proyecto, early_start[node1] + tiempos_proyecto[node1] + duracion_total_proyecto - late_start[node2]),
        'Left': max(duracion_total_proyecto, early_start[node2] + tiempos_proyecto[node2] + duracion_total_proyecto - late_start[node1])
    }
    mejor_opcion = min(D_max, key=D_max.get)

    # Agregar la nueva precedencia al grafo existente
    if mejor_opcion == 'Right':
        respuesta = f'{node1} -> {node2}'
        G_proyecto.add_edge(node1, node2, weight=tiempos_proyecto[node2])
    else:
        respuesta = f'{node2} -> {node1}'
        G_proyecto.add_edge(node2, node1, weight=tiempos_proyecto[node1])

    # Recalcular el camino crítico
    nuevo_camino_critico = nx.dag_longest_path(G_proyecto, weight='weight')
    nuevo_camino_critico = [nodo for nodo in nuevo_camino_critico if nodo not in [nodo_inicial_ficticio, nodo_final_ficticio]]
    nueva_duracion_total = sum(tiempos_proyecto[nodo] for nodo in nuevo_camino_critico)

    return nueva_duracion_total, respuesta,nuevo_camino_critico

# Ejemplo de llamada a la función con nodos diferentes
resultado_nuevo_camino = camino_nuevo('G', 'I')

# Print the results
print(f"4. Resultado del nuevo camino entre G e I: {resultado_nuevo_camino[1]}, Nueva duración total: {resultado_nuevo_camino[0]}, Nuevo camino crítico: {resultado_nuevo_camino[2]}")


1. Duración mínima del proyecto: 56 días
2. Camino Crítico: ['D', 'G', 'H', 'C', 'M', 'O', 'S', 'T', 'Q', 'U']
3. Suma de los márgenes de las actividades C, G, L y S: 9, Margenes:{'α': 0, 'A': 20, 'B': 20, 'C': 0, 'H': 0, 'D': 0, 'G': 0, 'E': 5, 'F': 5, 'I': 9, 'J': 9, 'K': 10, 'L': 9, 'M': 0, 'O': 0, 'P': 2, 'T': 0, 'Q': 0, 'R': 2, 'S': 0, 'U': 0, 'ω': 0} 
4. Resultado del nuevo camino entre G e I: G -> I, Nueva duración total: 56, Nuevo camino crítico: ['D', 'G', 'H', 'C', 'M', 'O', 'S', 'T', 'Q', 'U']
